# IST736 Text Mining - Project
## Martin Alonso - 2019-02-09

### Introduction
This project will focus on using home descriptions, number of rooms, and home location to predict the average price range for an Airbnb accomodations in Texas. 

#### Data
The data for this project was obtained from Kaggle (https://www.kaggle.com/PromptCloudHQ/airbnb-property-data-from-texas/home). It consists of 18,259 observations, each having ten variables. These variables include average rate per night, bedroom counts, city, date of listing, description, latitude, longitude, title, and url.  
The variable of interest - predictor variable - is the average rate per night, which will be converted into a range variable, so as to build a classifier model. Bedroom count, city, and date listing are single information variable; description and title have text information regarding the property and will be the main focus of the analysis and predictor model. Latitude and longitude will be associated with city, while url will be analyzed to see if it matches each property's title. 

#### Methodology 
Before building any model, the data set must be analyzed thouroughly to make sure that their is a relationship between the data and what is to be predicted: Namely, price range of a property.  
The first step will be loading and cleaning the data. After this is done, a new variable, price range, will be created based on the average rate per night variable. The reason this will be done is because we want to build a classifier model rather than a regression model.  
Once all this is done, the data will be explored, looking for trends between the different variables. Once this is completed, we'll vectorize the texts variables (description and title), remove stop words, stem and lemmatize the text, and perform both sentiment analysis as to whether the text is positive or negative, and explore the relationship between the most common words and location, price range, and number of bedrooms.  
Finally, once the data exploration is completed, three models will be built and compared, focusing on whether the text and the additional variables are good predictors of the price range of the property.  

#### Analysis

In [ ]:
# data exploration: eda price range by location, number of bedrooms, time series of avg price, relationship between bedrooms and locations, 
# number of listings by location per month

# word vectorization, stop word removal, lemming and stemming, most common words. 
# common words by price, room, location. 
# text sentimentality 

# feature engineering: price to price range

# nltk, nb, svm models 